In [27]:
#Problem: POS Machine, Improved
#My Solution
from tkinter import *
from tkinter import ttk, filedialog, Toplevel, messagebox
import pandas as pd
import os
window = Tk()
window.geometry('600x400')
window.title('PP-POS System')
class POSMachine:
    def __init__(self, main):
        fr_main = Frame(main)
        fr_main.pack()
        
        #instances
        self.__filename = ''
        self.__df_PriceList = ''
        self.__Quantity = ''
        self.__ProductID = ''
        self.__invoice = 'Sales Invoice\nID\tProduct\tPrice\tQuantity\tTotal'
        self.__LTotal = []
        self.__all = {}
        self.__counter = 1
        
        #Main Labels
        self.__lb_welcome = Label(fr_main, text = 'Welcome to PP-POS System.\nPlease choose an option.')
        self.__lb_pricelist = Label(fr_main, text = 'Price List File: ')
        
        #Buttons
        btn_setPriceList = Button(fr_main, text = 'Set Price List', command = self.setPriceList)
        btn_showPriceList = Button(fr_main, text = 'Show Price List', command = self.showPriceList)
        btn_makeNewTransaction = Button(fr_main, text = 'Make New Transaction', command = self.MakeNewTransaction)
        btn_createSalesReport = Button(fr_main, text = 'Sales Report', command = self.createSalesReport)
        btn_exit = Button(fr_main, text = 'Quit Program', command = self.exit)
        
        #Main Layout
        self.__lb_welcome.pack(fill = BOTH, side = TOP)
        self.__lb_pricelist.pack(side = TOP, anchor = 'w', pady = 10)
        btn_setPriceList.pack(side = LEFT, pady = 5, padx = 5)
        btn_showPriceList.pack(side = LEFT, pady = 5, padx = 5)
        btn_makeNewTransaction.pack(side = LEFT, pady = 5, padx = 5)
        btn_createSalesReport.pack(side = LEFT, pady = 5, padx = 5)
        btn_exit.pack(side = LEFT, pady = 5, padx = 5)
        
        #Transactions Frame
        self.__fr_transactions = Frame(main)
        self.__fr_transactions.pack(fill = BOTH)
        #Hide transaction frame until maketransaction button is clicked
        self.__fr_transactions.pack_forget() #https://www.geeksforgeeks.org/how-to-hide-recover-and-delete-tkinter-widgets/
        self.__lbf_transactions = LabelFrame(self.__fr_transactions, text = 'Input Product ID and Quantity')
        lb_ProductID = Label(self.__lbf_transactions, text = 'Product ID:')
        lb_Quantity = Label(self.__lbf_transactions, text = 'Quantity:')
        self.__entry_ProductID = Entry(self.__lbf_transactions, width = 10)
        self.__entry_Quantity = Entry(self.__lbf_transactions, width = 10)
        btn_submit = Button(self.__lbf_transactions, text = 'Submit', command = self.addProductToTransaction)
        btn_stop = Button(self.__lbf_transactions, text = 'End Transaction', command = self.createSalesInvoice)
        
        #Transaction layouting
        self.__lbf_transactions.pack(fill = BOTH, expand = True, pady = 10)
        lb_ProductID.grid(row = 0, column = 0)
        self.__entry_ProductID.grid(row = 0, column = 1)
        lb_Quantity.grid(row = 1, column = 0)
        self.__entry_Quantity.grid(row = 1, column = 1)
        btn_submit.grid(row = 2, column = 0)
        btn_stop.grid(row = 2, column = 1)

        #Viewing Frame
        self.__fr_view = Frame(main)
        self.__fr_view.pack(fill = BOTH)
        #frame hides until maketransaction is pressed
        self.__fr_view.pack_forget() #https://www.geeksforgeeks.org/how-to-hide-recover-and-delete-tkinter-widgets/ #hide until it gets called
        self.__lbf_view = LabelFrame(self.__fr_view, text = 'Transactions')
        
        #Main Treeview
        self.__tranl_view = ttk.Treeview(self.__lbf_view)
        verscrlbar = ttk.Scrollbar(self.__lbf_view, orient = 'vertical', command = self.__tranl_view.yview) #makes the scrollbar
        self.__tranl_view.configure(yscrollcommand = verscrlbar.set) #allows the treeview to access scrollbar
        verscrlbar.pack(side = 'right', fill = 'y')
        horscrlbar = ttk.Scrollbar(self.__lbf_view, orient = 'horizontal', command = self.__tranl_view.xview)
        self.__tranl_view.configure(xscrollcommand = horscrlbar.set)
        horscrlbar.pack(side = 'bottom', fill = 'x')
        
        #Viewing layouting for Treeview
        self.__lbf_view.pack(fill = BOTH, expand = True, pady = 10)
        self.__tranl_view.pack(fill = BOTH, expand = True)

    def setPriceList(self):
        self.__lb_welcome.configure(text = 'Please choose an option.')
        self.__filename = filedialog.askopenfilename(initialdir = os.getcwd(), title = 'Select a File', filetypes = (('All files', '*.*'), ('CSV Files', '*.csv'), ('Text Files', '*.txt'))) #https://www.geeksforgeeks.org/file-explorer-in-python-using-tkinter/?ref=lbp        
        if self.__filename[-4:] == '.txt':
            sep_perator = "\t"
        else:
            sep_perator = ','
        try:
            #checks the headers/first row to make sure it is a valid price list
            headrow = pd.read_csv(self.__filename, sep = sep_perator, nrows = 1)
            if 'ID' in headrow and 'Product' in headrow and 'Price' in headrow:
                self.__df_PriceList = pd.read_csv(self.__filename, sep = sep_perator)
                self.__lb_pricelist.configure(text = 'Price List: ' + str(self.__filename))
            else:
                messagebox.showerror('Error','Invalid price list.')
        except:
            messagebox.showerror('Error','Invalid file. Only .txt and .csv files are supported.')
        
    def showPriceList(self):
        self.__lb_welcome.configure(text = 'Please choose an option.')
        if self.__filename != '':
            #open new window of price list
            newWindow_PriceList = Toplevel() #https://www.geeksforgeeks.org/open-a-new-window-with-a-button-in-python-tkinter/?ref=lbp
            newWindow_PriceList.title('Price List')
            newWindow_PriceList.geometry('600x400')
            
            #Treeview for Price List
            tab_view = ttk.Treeview(newWindow_PriceList) #https://www.geeksforgeeks.org/python-tkinter-treeview-scrollbar/?ref=lbp
            
            # vertical scroll bar
            verscrlbar = ttk.Scrollbar(newWindow_PriceList, orient = 'vertical', command = tab_view.yview)
            tab_view.configure(yscrollcommand = verscrlbar.set) #allows the treeview to access scrollbar
            verscrlbar.pack(side = 'right', fill = 'y')
            
            #horizontal scroll bar
            horscrlbar = ttk.Scrollbar(newWindow_PriceList, orient = 'horizontal', command = tab_view.xview)
            tab_view.configure(xscrollcommand = horscrlbar.set)
            horscrlbar.pack(side = 'bottom', fill = 'x')
            
            #headers
            tab_view['column'] = list(self.__df_PriceList.columns) #gets the headers of each column
            tab_view['show'] = 'headings' #shows the headers
            for head in tab_view['column']: #adds the headers into the treeview
                tab_view.heading(head, text = head)
            
            #insert rows of data into treeview
            for index, row in self.__df_PriceList.iterrows(): #https://www.geeksforgeeks.org/different-ways-to-iterate-over-rows-in-pandas-dataframe/
                tab_view.insert('', 'end', values = row.tolist()) #https://docs.python.org/3/library/tkinter.ttk.html#treeview 
                #adds from the top level and at the end/bottom of every row
            tab_view.pack(fill = BOTH, expand = True)
        else:
            messagebox.showerror('Error', 'Please set a price list first.')
    
    def MakeNewTransaction(self):
        #resets instances for new transactions
        self.__invoice = 'Sales Invoice\nID\tProduct\tPrice\tQuantity\tTotal'
        self.__LTotal.clear()
        self.__lb_welcome.configure(text = 'Please choose an option.')

        #resets treeview for new transactions
        if self.__tranl_view.get_children():
            for row in self.__tranl_view.get_children():
                self.__tranl_view.delete(row)

        #Checks if price list has already been set
        if self.__filename != '':
            self.__fr_view.pack_forget() #hides it again to make sure that fr_transactions frame comes first before fr_view frame
            self.__fr_transactions.pack(fill = BOTH)
            self.__fr_view.pack(fill = BOTH)
        
        else:
            messagebox.showerror('Error', 'Please set a price list first.')

    def addProductToTransaction(self):
        try:
            #Deletes the last (Total) row to prevent duplication
            if self.__tranl_view.get_children():
                lastrowvalue = self.__tranl_view.item(self.__tranl_view.get_children()[-1], 'values') #https://docs.python.org/3/library/tkinter.ttk.html#ttk-treeview
                #Makes sure that the last row is a total row and not product transaction
                if lastrowvalue[3] == 'Total':
                    self.__tranl_view.delete(self.__tranl_view.get_children()[-1])
                    
            #turns string input to int
            self.__ProductID = int(self.__entry_ProductID.get())
            self.__Quantity = int(self.__entry_Quantity.get())

            if self.__ProductID > 0 and self.__Quantity > 0: #Makes sure that input is > 0
                ptt_df = self.__df_PriceList.set_index('ID') #setting ID as index local to this function           
                self.__LTotal.append(ptt_df.loc[self.__ProductID]['Price'] * self.__Quantity)
                
                #clears input for next transaction
                self.__entry_ProductID.delete(0, END) #https://www.tutorialspoint.com/how-to-clear-the-entry-widget-after-a-button-is-pressed-in-tkinter#:~:text=We%20can%20clear%20the%20content,by%20creating%20a%20Button%20Object.
                self.__entry_Quantity.delete(0, END)
                
                #edits the treeview to add new headers dn transactions
                self.__tranl_view['columns'] = ('ID', 'Product', 'Price', 'Quantity', 'Amount')
                self.__tranl_view['show'] = 'headings'
                for col in self.__tranl_view['columns']:
                    self.__tranl_view.column(col, width = 80) #cahnges every column to width 80
                for head in self.__tranl_view['columns']:
                    self.__tranl_view.heading(head, text = head) #puts all the headers into the columns
                
                #Inserts transaction and running total into the Treeview
                self.__tranl_view.insert('', 'end', values = (self.__ProductID, 
                                                              ptt_df.loc[self.__ProductID]['Product'], 
                                                              ptt_df.loc[self.__ProductID]['Price'], 
                                                              self.__Quantity, 
                                                              ptt_df.loc[self.__ProductID]['Price'] * self.__Quantity))
                self.__tranl_view.insert('', 'end', values = ('', '', '', 'Total', sum(self.__LTotal)))
                
                #creates the string for the .txt file invoice
                self.__invoice = self.__invoice + '\n{id}\t{product}\t{price}\t{quantity}\t{total}'.format(id = self.__ProductID,
                                                                                     product = ptt_df.loc[self.__ProductID]['Product'],
                                                                                     price = ptt_df.loc[self.__ProductID]['Price'],
                                                                                     quantity = self.__Quantity,
                                                                                     total = ptt_df.loc[self.__ProductID]['Price'] * self.__Quantity)
                #creates the dictionary for the sales report
                if ptt_df.loc[self.__ProductID]['Product'] not in self.__all: #adds the product if it is still not in dictionary
                    self.__all[ptt_df.loc[self.__ProductID]['Product']] = [ptt_df.loc[self.__ProductID]['Product'], ptt_df.loc[self.__ProductID]['Price'], self.__Quantity, ptt_df.loc[self.__ProductID]['Price'] * self.__Quantity]
                
                else: #edits values in existing keys in the dictionary accordingly
                    self.__all[ptt_df.loc[self.__ProductID]['Product']][1] = ptt_df.loc[self.__ProductID]['Price']
                    self.__all[ptt_df.loc[self.__ProductID]['Product']][2] += self.__Quantity
                    self.__all[ptt_df.loc[self.__ProductID]['Product']][3] += ptt_df.loc[self.__ProductID]['Price'] * self.__Quantity
            else:
                messagebox.showerror('Error', 'Product not found or invalid input.')
                self.__entry_ProductID.delete(0, END) #clears the entries
                self.__entry_Quantity.delete(0, END)
        except:
            messagebox.showerror('Error', 'Product not found or invalid input.')
            self.__entry_ProductID.delete(0, END) 
            self.__entry_Quantity.delete(0, END)
            self.__tranl_view.insert('', 'end', values = ('', '', '', 'Total', sum(self.__LTotal))) #making sure that it does not delete the total bar when there is an error
        
    def createSalesInvoice(self):
        self.__fr_transactions.pack_forget() #hides the frame to prevent more transactions
        with open('invoice{}.txt'.format(self.__counter), 'w') as f: #opens and closes file
            f.write(self.__invoice)
            f.write('\n\t\t\tGrand Total\t{}'.format(sum(self.__LTotal)))
        self.__counter += 1
        self.__lbf_view.configure(text = 'Sales Invoice')
        
    def createSalesReport(self):
        self.__lb_welcome.configure(text = 'Please choose an option.')
        
        if self.__all != {}: #Making sure that at least one transaction has happened
            
            #Clear Previous contents
            if self.__tranl_view.get_children():
                for row in self.__tranl_view.get_children():
                    self.__tranl_view.delete(row)
            
            #hide the transactions frame to prevent transactions
            self.__fr_transactions.pack_forget()
    
            #converts the dict to a csv to be read by treeview
            dfSalesReport = pd.DataFrame.from_dict(self.__all, orient = 'index', columns = ['Product', 'Price', 'Total Number Sold', 'Total Sales']) #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_dict.html
    
            #change the treeview to sales report
            self.__lbf_view.configure(text = 'Sales Report')
            self.__tranl_view['column'] = list(dfSalesReport.columns) #specifies the headers
            self.__tranl_view['show'] = 'headings' #only show the specified headings
            for head in self.__tranl_view['column']: #adds the headings into the treeview
                self.__tranl_view.heading(head, text = head)
            
            #insert rows of data into treeview
            for index, row in dfSalesReport.iterrows(): #https://www.geeksforgeeks.org/different-ways-to-iterate-over-rows-in-pandas-dataframe/
                self.__tranl_view.insert('', 'end', values = row.tolist())
        else:
            messagebox.showerror('Error', 'Please make a transaction first.')
        
    def exit(self):
        self.__lb_welcome.configure(text = 'Please choose an option.')
        response = messagebox.askokcancel('Exit Program', 'Are you sure you want to close the program?')
        if response:
            window.destroy() #https://www.geeksforgeeks.org/destroy-method-in-tkinter-python/

POSMachine(window)
window.mainloop()

In [ ]:
'''
I hereby attest to the truth of the following facts:

I have not discussed the Python code in my program with anyone other than my instructor or the teaching assistants assigned to this course.

I have not used Python code obtained from another student, or any other unauthorized source, whether modified or unmodified.

If any Python code or documentation used in my program was obtained from another source, it has been clearly noted with citations in the comments of my program.
'''